In [ ]:
# autoreload
%reload_ext autoreload
%autoreload 2

!git clone https://github.com/shiukaheng/minGS.git
#!git clone https://github.com/nadavo11/Aqua-Splat-Aquatic-3D-Novel-View-synthesis-Via-in-scene-deflection.git

!pip install plyfile lpips viser requests imageio matplotlib --q

# make sure we have a compiler helper
!pip install ninja  # speeds up CUDA builds

# --- diff-gaussian-rasterization
!pip install -e ./minGS/submodules/diff-gaussian-rasterization

# --- simple-knn
!pip install -e ./minGS/submodules/simple-knn

 # system packages
! sudo apt install ffmpeg colmap

! pip install pycolmap

# python env (same one you train GS in)
! pip install numpy pillow tqdm

!sudo apt update && sudo apt install xvfb mesa-utils libgl1-mesa-dri -y

!pip install pycolmap

!pip install -e ./minGS/submodules/diff-gaussian-rasterization

import sys
sys.path.append('/content/minGS')
sys.path.append('/content/minGS/submodules/diff-gaussian-rasterization')
sys.path.append('/content/minGS/submodules/simple-knn')
sys.path.append('/content/minGS')

# aquatic_gs_baseline.py – baseline Gaussian Splatting demo (using Tanks&Temples 'Barn' COLMAP dataset)
# Automatically install missing dependencies and run Gaussian Splatting demo for 'Barn'.

# ------- Dependencies installation (run once at start) ------------------------
import importlib
import subprocess
import sys

# List of PyPI packages and minGS repo to ensure installed
REQUIRED_PKGS = [
    "git+https://github.com/shiukaheng/minGS.git",  # Gaussian Splatting core
    "plyfile",                                       # COLMAP PLY support
    "lpips",                                         # perceptual loss
    "viser",                                         # visualization helper
    "requests",                                      # HTTP
    "imageio",                                       # image I/O
    "matplotlib"                                     # plotting
]

for pkg in REQUIRED_PKGS:
    name = pkg.split("#")[0].strip()
    try:
        if name.startswith("git+"):
            importlib.import_module("gs")
        else:
            importlib.import_module(name)
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])



fatal: destination path 'minGS' already exists and is not an empty directory.
Obtaining file:///content/minGS/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
  Attempting uninstall: diff_gaussian_rasterization
    Found existing installation: diff_gaussian_rasterization 0.0.0
    Uninstalling diff_gaussian_rasterization-0.0.0:
      Successfully uninstalled diff_gaussian_rasterization-0.0.0
  Running setup.py develop for diff_gaussian_rasterization
Obtaining file:///content/minGS/submodules/simple-knn
  Preparing metadata (setup.py) ... done
  Attempting uninstall: simple_knn
    Found existing installation: simple_knn 0.0.0
    Uninstalling simple_knn-0.0.0:
      Successfully uninstalled simple_knn-0.0.0
  Running setup.py develop for simple_knn
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
colmap is already the newest version (3.7-2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 


## Baseline script to train **minGS** on a small COLMAP dataset.
- installs the minimal 3‑D Gaussian Splatting implementation
- downloads a tiny demo dataset (bonsai) if `DATA_DIR` is empty
- runs a very short training loop (2 k iters)
- dumps preview renderings every 100 steps
- saves the final model to `aquatic_baseline_gs.pth`

You can use the same scaffolding later and simply swap the `GaussianModel.forward` call with your deflection‑aware version.


# data

In [ ]:
from huggingface_hub import snapshot_download

local_path = snapshot_download(
    repo_id="nadavo11/red_truck_GS",
    repo_type="dataset",
    local_dir="/content/red_truck_GS",    # any path you like
    resume_download=True                      # skip files already on disk
)
print("Dataset at:", local_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 1334 files:   0%|          | 0/1334 [00:00<?, ?it/s]

val.txt:   0%|          | 0.00/511 [00:00<?, ?B/s]

train.txt:   0%|          | 0.00/4.69k [00:00<?, ?B/s]

Dataset at: /content/red_truck_GS


## split train and test

In [ ]:
import random
from pathlib import Path
from gs.io.colmap import load

# ------------------------------------------------------------------
# 1. load every COLMAP view
# ------------------------------------------------------------------
DATA_DIR = Path("red_truck_GS")
cameras, pointcloud = load(str(DATA_DIR))

all_cams = list(cameras)          # list[BaseCamera]

# ------------------------------------------------------------------
# 2. SPLIT  — 90 % train / 10 % val  (or whatever ratio you like)
# ------------------------------------------------------------------
random.seed(42)                             # reproducible split
random.shuffle(all_cams)

val_frac   = 0.10                           # ← tweak here
n_val      = max(1, int(len(all_cams)*val_frac))
val_cams   = all_cams[:n_val]
train_cams = all_cams[n_val:]

print(f"{len(train_cams)} train views   |   {len(val_cams)} val views")


67 train views   |   7 val views


In [ ]:
# split_red_truck.py
from pathlib import Path
import random, json

root      = Path("red_truck_GS")
img_dir   = root / "images"          # where COLMAP put the JPG/PNG frames
split_dir = root / "splits"          # just text files with the frame ids
split_dir.mkdir(exist_ok=True)

all_imgs  = sorted(img_dir.glob("*"))      # 000001.png, 000002.png …
random.seed(42)
random.shuffle(all_imgs)

val_frac = 0.1
n_val    = max(1, int(len(all_imgs)*val_frac))
val_imgs, train_imgs = all_imgs[:n_val], all_imgs[n_val:]

# write text files (handy for other scripts)
(split_dir / "train.txt").write_text("\n".join(p.name for p in train_imgs))
(split_dir / "val.txt").write_text("\n".join(p.name for p in val_imgs))

print(f"train {len(train_imgs)}  |  val {len(val_imgs)}")


train 293  |  val 32


In [ ]:
# link_views.py
from pathlib import Path, os

root = Path("red_truck_GS")
for split in ("train", "val"):
    (root / split / "images").mkdir(parents=True, exist_ok=True)

for split in ("train", "val"):
    for name in (root / "splits" / f"{split}.txt").read_text().splitlines():
        src = root / "images" / name
        dst = root / split / "images" / name
        if not dst.exists():
            os.link(src, dst)      # hard-link → 0-byte overhead


In [ ]:

# ------- Imports --------------------------------------------------------------
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement

from gs.core.GaussianModel import GaussianModel
from gs.io.colmap import load
from gs.helpers.loss import l1_loss

url = "https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/datasets/input/tandt_db.zip"
ROOT = Path("./data")
DATA_NAME = "tandt"
DATA_DIR = Path(f"./data/{DATA_NAME}/train")
# ------- Prepare dataset ------------------------------------------------------
# We'll download the authors' Tanks&Temples+DeepBlending COLMAP archive and extract only the 'tandt' scene
from baseline_utils.utils import *
get_data_from_web(url, ROOT, DATA_NAME)


# ------- Load COLMAP data -----------------------------------------------------
# Expects: BARN_DIR/sparse/0/{cameras.bin, images.bin, points3D.bin}, BARN_DIR/images/*
cameras, pointcloud = load(str(DATA_DIR))
print(f"Loaded {len(cameras)} cameras and {len(pointcloud.points)} sparse points from Barn")


# ------- Build Gaussian Splatting model ---------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GaussianModel.from_point_cloud(pointcloud).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, eps=1e-15)

# ------- Quick training loop --------------------------------------------------
NUM_ITERS = 20000  # keep small for baseline; bump for quality
LOG_EVERY = 1000

for it in range(NUM_ITERS):
    cam = cameras[it % len(cameras)].to(device)
    rgb_pred = model(cam)
    loss = l1_loss(rgb_pred, cam.image.to(device))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)

    if (it + 1) % LOG_EVERY == 0 or it == 0:
        print(f"iter {it+1:4d}/{NUM_ITERS} | L1 = {loss.item():.4f}")
        vis = (rgb_pred.clamp(0, 1).permute(1, 2, 0).detach().cpu().numpy() * 255).astype('uint8')
        torch.save(model.state_dict(), "aquatic_baseline_gs.pth")


        imageio.imwrite(f"preview_{it+1:04d}.png", vis)

# ------- Save model -----------------------------------------------------------
torch.save(model.state_dict(), "aquatic_baseline_gs.pth")
print("Training finished – previews and checkpoint written.")


ModuleNotFoundError: No module named 'baseline_utils'

#nice visuals

In [ ]:
import numpy as np
import torch
from gs.core.BaseCamera import BaseCamera

# --- add a drop-in replacement ------------------------------------------------
def _look_at(
        eye: torch.Tensor,          # (3,)  camera position
        center: torch.Tensor,       # (3,)  look-at point
        up: torch.Tensor,           # (3,)  world-up
        width: int, height: int,    # image size (px)
        fx: float                   # focal length in px (assume square pixels)
):
    eye_np, center_np, up_np = map(lambda v: v.detach().cpu().numpy(), (eye, center, up))

    # Build rotation (camera → world) with the usual OpenGL look-at convention
    f = center_np - eye_np
    f /= np.linalg.norm(f)
    s = np.cross(f, up_np)
    s /= np.linalg.norm(s)
    u = np.cross(s, f)

    # Camera‐to-world (R^T | eye); we need world-to-camera, so take the transpose.
    R_c2w = np.stack([s, u, -f], axis=0)          # shape (3, 3)
    R = R_c2w.T                                   # world → camera
    t = -R @ eye_np                               # translation in camera frame

    fov_x = 2 * np.arctan(width  / (2 * fx))
    fov_y = 2 * np.arctan(height / (2 * fx))

    return BaseCamera(
        image_height=height,
        image_width=width,
        fov_x=float(fov_x),
        fov_y=float(fov_y),
        R=R.astype(np.float32),
        t=t.astype(np.float32),
    )

# attach it so the rest of the code can stay identical
BaseCamera.look_at = staticmethod(_look_at)
# ------------------------------------------------------------------------------



In [ ]:
import numpy as np
import torch
from gs.core.BaseCamera import BaseCamera

def orbit_around_camera(
        ref_cam: BaseCamera,
        radius: float,
        n_views: int = 120,
        axis: str = "yaw",            # "yaw"  (spin left/right)
                                        # "pitch" (tilt up/down)
                                        # "roll"  (spin around optical axis)
        up_hint: torch.Tensor | None = None,   # world-space up  (optional)
        width: int = 800,
        height: int = 800,
        fx: float = 700,
) -> list[BaseCamera]:

    # -- extract the reference-camera transforms -----------------------------
    R_wc = ref_cam.world_view_transform[:3, :3].T      # world←camera
    t_wc = ref_cam.camera_center                       # camera position (world)

    # local basis: x = right, y = up, z = -view
    axis_vectors = {
        "yaw":   torch.tensor([0, 1, 0]).to(device),   # rotate around local Y-up
        "pitch": torch.tensor([1, 0, 0]).to(device),   # rotate around local X-right
        "roll":  torch.tensor([0, 0, 1]).to(device),   # rotate around local Z-view
    }
    a_local = axis_vectors[axis].float()

    # up vector for look_at; fall back to camera’s own +Y if none supplied
    up_world = up_hint if up_hint is not None else R_wc @ torch.tensor([0, 1, 0],dtype=torch.float).to(device)

    cams = []
    for theta in torch.linspace(1.2*np.pi, 1.8*np.pi, n_views):
        # rotation matrix in *camera* space, then send to world space
        cos_t, sin_t = torch.cos(theta), torch.sin(theta)
        if axis == "yaw":
            R_delta = torch.tensor([[ cos_t, 0, sin_t],
                                    [     0, 1,     0],
                                    [-sin_t, 0, cos_t]]).to(device)
        elif axis == "pitch":
            R_delta = torch.tensor([[1,     0,      0],
                                    [0, cos_t, -sin_t],
                                    [0, sin_t,  cos_t]]).to(device)
        else:  # roll
            R_delta = torch.tensor([[cos_t, -sin_t, 0],
                                    [sin_t,  cos_t, 0],
                                    [    0,      0, 1]]).to(device)

        # eye position: start at (radius,0,0) in ref-cam space, rotate, go world
        eye_local  = torch.tensor([radius, 0, 0]).to(device)
        eye_world  = (R_wc @ (R_delta @ eye_local)) + t_wc

        # look-at point = pivot itself (camera centre)
        cams.append(
            BaseCamera.look_at(
                eye_world, t_wc, up_world,
                width=width, height=height, fx=fx
            ).to("cuda")
        )

    return cams


# get the video

In [ ]:
# choose any dataset camera you like (e.g. index 0)
ref_cam = cameras[2]
device ="cuda"
# build an orbit that yaws 360° around that camera
cams = orbit_around_camera(
    ref_cam,
    radius=2.0,      # metres in *camera* space
    n_views=120,
    axis="yaw",       # try "pitch" or "roll" for other motions
    width=800, height=800, fx=700
)

# render exactly as before
frames = []
for cam in cams:
    with torch.no_grad():
        img = model(cam).clamp(0,1).permute(1,2,0).cpu().numpy()
    frames.append((img*255).astype("uint8"))

imageio.mimsave("pivot_turntable.mp4", frames, fps=24)
print("Saved pivot_turntable.mp4")


____________

# make our own aquatic data!

In [ ]:
from baseline_utils.make_dataset import *

In [ ]:
!colmap feature_extractor --help | grep gpu


In [ ]:
!xvfb-run colmap gui


In [ ]:
! python /content/Aqua-Splat-Aquatic-3D-Novel-View-synthesis-Via-in-scene-deflection/baseline_utils/make_dataset.py /content/Aqua-Splat-Aquatic-3D-Novel-View-synthesis-Via-in-scene-deflection//baseline_utils/vid.MOV --k 120 --fps 12 --overlap 5 --dataset red_truck_GS/train/ --from_frames


## push to hf

In [ ]:
!pip install -q --upgrade huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import upload_folder
api = HfApi()


upload_folder(
    repo_id="nadavo11/red_truck_GS",
    folder_path="red_truck_GS",         # the root we generated
    repo_type="dataset",                    # same as above
    path_in_repo="",                        # keep structure as-is

    ignore_patterns=["*.db"]                # skip the COLMAP DB if you like
)

# train

In [ ]:
!curl -X GET \
     "https://datasets-server.huggingface.co/rows?dataset=nadavo11%2Faquarium-gaussian-splats&config=default&split=train&offset=0&length=100"

In [ ]:
# ------- Imports --------------------------------------------------------------
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement

from gs.core.GaussianModel import GaussianModel
from gs.io.colmap import load
from gs.helpers.loss import l1_loss


ROOT = Path("/content/")
DATA_NAME = "aquarium"
DATA_DIR = Path(f"{DATA_NAME}")
# ------- Load COLMAP data -----------------------------------------------------
# Expects: BARN_DIR/sparse/0/{cameras.bin, images.bin, points3D.bin}, BARN_DIR/images/*
cameras, pointcloud = load(str(DATA_DIR))
print(f"Loaded {len(cameras)} cameras and {len(pointcloud.points)} sparse points from Barn")


# ------- Build Gaussian Splatting model ---------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GaussianModel.from_point_cloud(pointcloud).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, eps=1e-15)

#save as .ply
model.save_ply("./aquarium_gaussians.ply",)         # ← one-liner

# ------- Quick training loop --------------------------------------------------
NUM_ITERS = 20000  # keep small for baseline; bump for quality
LOG_EVERY = 1000

for it in range(NUM_ITERS):
    cam = cameras[it % len(cameras)].to(device)
    rgb_pred = model(cam)
    loss = l1_loss(rgb_pred, cam.image.to(device))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)

    if (it + 1) % LOG_EVERY == 0 or it == 0:
        print(f"iter {it+1:4d}/{NUM_ITERS} | L1 = {loss.item():.4f}")
        vis = (rgb_pred.clamp(0, 1).permute(1, 2, 0).detach().cpu().numpy() * 255).astype('uint8')
        torch.save(model.state_dict(), "aquatic_baseline_gs.pth")
        model.save_ply("./aquarium_gaussians.ply")         # ← one-liner



        imageio.imwrite(f"preview_{it+1:04d}.png", vis)

# ------- Save model -----------------------------------------------------------
torch.save(model.state_dict(), "aquatic_baseline_gs.pth")
print("Training finished – previews and checkpoint written.")


In [ ]:
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement

from gs.core.GaussianModel import GaussianModel
from gs.io.colmap import load
from gs.helpers.loss import l1_loss


ROOT = Path("/content/")
DATA_NAME = "aquarium"
DATA_DIR = Path(f"{DATA_NAME}")
# ------- Load COLMAP data -----------------------------------------------------
# Expects: BARN_DIR/sparse/0/{cameras.bin, images.bin, points3D.bin}, BARN_DIR/images/*
cameras, pointcloud = load(str(DATA_DIR))

# push to hf

In [ ]:
!pip install -q --upgrade huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="nadavo11/aquarium",
    folder_path="aquarium",         # the root we generated
    repo_type="dataset",                    # same as above
    path_in_repo="",                        # keep structure as-is
    ignore_patterns=["*.db"]                # skip the COLMAP DB if you like
)

# # add the checkpoints too
# upload_folder(
#     repo_id="nadavo11/aquarium",
#     folder_path=".",                        # current dir
#     repo_type="dataset",
#     path_in_repo="checkpoints",
#     #include_patterns=["*.pth", "*.ply"]
# )


# retrieve from hf

In [ ]:
# get dataset from hugging face
from huggingface_hub import snapshot_download

local_path = snapshot_download(
    repo_id="nadavo11/aquarium",
    repo_type="dataset",
    local_dir="aquarium_downloaded",  # local target folder
    local_dir_use_symlinks=False      # ensure real files are copied
)

print(f"Dataset downloaded to: {local_path}")

# Baseline Train

In [ ]:
from gs.trainers.basic import *

In [ ]:
# ------- Imports --------------------------------------------------------------
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement

from gs.core.GaussianModel import GaussianModel
from gs.io.colmap import load
from gs.helpers.loss import l1_loss


ROOT = Path("/content/")
DATA_NAME = "red_truck_GS"
DATA_DIR = "red_truck_GS"
# ------- Load COLMAP data -----------------------------------------------------
# Expects: BARN_DIR/sparse/0/{cameras.bin, images.bin, points3D.bin}, BARN_DIR/images/*
cameras, pointcloud = load(str(DATA_DIR))
print(f"Loaded {len(cameras)} cameras and {len(pointcloud.points)} sparse points from Barn")
model = GaussianModel.from_point_cloud(pointcloud)

In [ ]:

#public = ngrok.connect(8088, proto="http")   # forwards 8080 → HTTPS URL
from gs.trainers.basic import *

train(model = model,
      cameras = train_cams,
      iterations = 10000,
      densify_until_iter = 10
      )

In [ ]:
model.save_ply("./latest_model.ply")


In [ ]:
import torch


from gs.core.GaussianModel import GaussianModel
from gs.visualization.Viewer import Viewer    # 👈
#from pyngrok import ngrok
#from pyngrok import ngrok


viewer = Viewer(model, frame_rate=15, width=1280,)

viewer.render_loop()

In [ ]:
#!pip install -q pyngrok
!ngrok config add-authtoken 2wVlctr7rHbnaxqVEokp84Jwsbh_4nFBXYEQQ8exfAYNPfL8Q


In [ ]:
torch.save(model.cpu(), "aquarium.pt")   # one line


In [ ]:
import torch
model2 = torch.load("aquarium.pt", weights_only=False)
model2.eval()

In [ ]:
# save to ply
model2.save_ply("./aquarium.ply")

In [ ]:
from google.colab import files

files.download("aquarium.pt")    # or "aquatic_baseline_gs.pth"


# *baseline*


## data

In [ ]:

# get dataset from hugging face
from huggingface_hub import snapshot_download

local_path = snapshot_download(
    repo_id="nadavo11/red_truck_GS",
    repo_type="dataset",
    local_dir="aquarium_downloaded",  # local target folder
    local_dir_use_symlinks=False      # ensure real files are copied
)

print(f"Dataset downloaded to: {local_path}")

## load train set

In [ ]:
from pathlib import Path
from gs.io.colmap import load

root = Path("red_truck_GS")            # repo you cloned from HF

# 1. load every view
all_cams, pointcloud = load(str(root))       # key = image_id, value = BaseCamera

# 2. read the split lists
train_ids = set((root/"splits/train.txt").read_text().splitlines())
val_ids   = set((root/"splits/val.txt").read_text().splitlines())


train_cams = [cam for cam in all_cams if cam.image_path.split('/')[-1] in train_ids]
val_cams   = [cam for cam in all_cams if cam.image_path.split('/')[-1] in val_ids]

print(f"{len(train_cams)} train cams | {len(val_cams)} val cams")

## train

In [ ]:
model = GaussianModel.from_point_cloud(pointcloud)
#public = ngrok.connect(8088, proto="http")   # forwards 8080 → HTTPS URL
from gs.trainers.basic import *

model = GaussianModel.from_point_cloud(pointcloud)
#public = ngrok.connect(8088, proto="http")   # forwards 8080 → HTTPS URL
from gs.trainers.basic import *

train(model = model,
      cameras = train_cams,
      iterations = 10000,
      )

In [ ]:
# ------- Imports --------------------------------------------------------------
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement

from gs.core.GaussianModel import GaussianModel
from gs.io.colmap import load
from gs.helpers.loss import l1_loss



In [ ]:
model = GaussianModel.from_point_cloud(pointcloud)
#public = ngrok.connect(8088, proto="http")   # forwards 8080 → HTTPS URL
from gs.trainers.basic import *

train(model = model,
      cameras = cameras,
      iterations = 100000,
      )

In [ ]:
model.eval()

In [ ]:
import torch, numpy as np
from plyfile import PlyData, PlyElement
from pathlib import Path
from gs.helpers.math import inverse_sigmoid   # already used in your code

# ----------------------------------------------------------------------
def add_plane_gaussians_and_save(model,
                                 out_ply="with_plane_markers.ply",
                                 rect_side_fraction=0.3,
                                 marker_scale=0.05,
                                 marker_opacity=0.9):
    """
    Append 6 red 'indicator' Gaussians (4 rectangle corners + 2 arrow points)
    that visualise the global plane stored in model.plane_p / model.plane_n,
    then save everything as one PLY file.

    Parameters
    ----------
    rect_side_fraction : float   size of square marker vs. scene diagonal
    marker_scale       : float   isotropic std-dev [world units]
    marker_opacity     : float   [0–1] alpha of the red blobs
    """
    device = model.positions.device
    dtype  = model.positions.dtype
    N      = len(model)

    # 1. build 6 indicator positions ---------------------------------
    p0   = model.plane_p.detach().cpu().numpy()
    nhat = model.plane_n.detach().cpu().numpy()

    # two orthonormal in-plane vectors
    u = np.cross([1, 0, 0], nhat); u /= np.linalg.norm(u)
    v = np.cross(nhat, u)
    side = rect_side_fraction * np.linalg.norm(model.positions.detach().cpu().numpy().ptp(0))

    rect = np.array([ p0 + side*(+u+v),
                      p0 + side*(+u-v),
                      p0 + side*(-u-v),
                      p0 + side*(-u+v) ])          # (4,3)
    arrow = np.vstack([p0, p0 + nhat*0.5*side])     # (2,3)
    P_ind = np.vstack([rect, arrow])                # (6,3)

    # 2. create Gaussian parameters for the indicators ---------------
    M = P_ind.shape[0]
    pos_t   = torch.tensor(P_ind, dtype=dtype, device=device)
    rot_t   = torch.zeros((M,4), dtype=dtype, device=device); rot_t[:,0] = 1
    sca_t   = torch.log(torch.full((M,3), marker_scale, dtype=dtype, device=device))
    opa_t   = inverse_sigmoid(torch.full((M,1), marker_opacity, dtype=dtype, device=device))

    # SH coefficients: DC term = bright red, others zero
    D = (model.sh_degree + 1) ** 2
    sh_t = torch.zeros((M, D, 3), dtype=dtype, device=device)
    sh_t[:,0,0] = 1.0   # red channel DC

    # 3. concatenate to the model ------------------------------------
    model.positions            = nn.Parameter(torch.cat([model.positions, pos_t], 0))
    model.rotations            = nn.Parameter(torch.cat([model.rotations, rot_t], 0))
    model.scales               = nn.Parameter(torch.cat([model.scales,    sca_t], 0))
    model.opacities            = nn.Parameter(torch.cat([model.opacities, opa_t], 0))
    model.sh_coefficients_0    = nn.Parameter(torch.cat([model.sh_coefficients_0,
                                                         sh_t[:,:1,:]], 0))
    model.sh_coefficients_rest = nn.Parameter(torch.cat([model.sh_coefficients_rest,
                                                         sh_t[:,1:,:]], 0))
    # etas—keep same size by duplicating last value
    model.etas                 = nn.Parameter(torch.cat([model.etas,
                                                         model.etas[-1:].repeat(M,1)], 0))

    # 4. write a single PLY (reuse existing save_ply method) ----------
    model.save_ply(out_ply)
    print(f"✓ Augmented model + plane markers written to {Path(out_ply).resolve()}")
model.save_ply("./model.ply")


In [ ]:
# autoreload
%autoreload
from gs.core.GaussianModel import GaussianModel

model2 = GaussianModel.from_point_cloud(pointcloud=pointcloud)

# get from ply

In [ ]:
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement
from gs.core.GaussianModel import GaussianModel

model = GaussianModel.from_ply("./model_19000.ply")

In [ ]:
from gs.io.colmap import load

# ------- Load COLMAP data -----------------------------------------------------
# Expects: BARN_DIR/sparse/0/{cameras.bin, images.bin, points3D.bin}, BARN_DIR/images/*
cameras, pointcloud = load(str("red_truck_GS"))
print(f"Loaded {len(cameras)} cameras and {len(pointcloud.points)} sparse points from red_truck_GS")



In [ ]:
from gs.trainers.basic import *
model = GaussianModel.from_point_cloud(pointcloud)
project="gs-red-truck"
run = wandb.init(project=project, resume="allow")

train(model = model,
      cameras = train_cams,
      iterations = 50000,
      val_cams = val_cams,
      run = run
      )

╭──────────────── viser ────────────────╮
│             ╷                         │
│   HTTP      │ http://localhost:8087   │
│   Websocket │ ws://localhost:8087     │
│             ╵                         │
╰───────────────────────────────────────╯

Loss: 0.10523281991481781:   3%|▎         | 1499/50000 [00:48<24:58, 32.38it/s]

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth


Loss: 0.1315668225288391:   3%|▎         | 1503/50000 [00:54<5:25:19,  2.48it/s]

✅ logged iter 1500 to W&B


...

Loss: 0.03001699224114418:  99%|█████████▉| 49499/50000 [46:04<00:29, 16.75it/s] 

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth


Loss: 0.025992782786488533:  99%|█████████▉| 49503/50000 [46:11<06:51,  1.21it/s]

✅ logged iter 49500 to W&B


Loss: 0.022683579474687576: 100%|██████████| 50000/50000 [46:41<00:00, 17.85it/s]


this is the train validation step

In [ ]:
model.save_ply(f"./model_final.ply")

In [ ]:
from google.colab import files

files.download("./model_37500.ply")    # or "aquatic_baseline_gs.pth"

In [ ]:
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement
from gs.core.GaussianModel import GaussianModel

model = GaussianModel.from_ply("./model_37500.ply").to("cuda")

In [ ]:
import torch, lpips
print(torch.cuda.is_available())          # → True
print(torch.version.cuda)                 # e.g. 12.1
print(torch.cuda.get_device_name(0))      # e.g. Tesla T4 / A100

In [ ]:
#=============================================================
#
#             validations & saving
#
#=============================================================
with torch.no_grad():
  rendered = model.forward(val_cams[0])
#model.save_ply(f"./model_{i}.ply")
vis = (rendered.clamp(0, 1).permute(1, 2, 0).detach().cpu().numpy() * 255).astype('uint8')
imageio.imwrite(f"preview_{i:04d}.png", vis)

In [ ]:
model2 = GaussianModel.from_point_cloud(pointcloud=pointcloud).to("cuda")

# Evaluation

In [ ]:
!pip install --upgrade wandb
!wandb login  # paste your API key or run in a browser

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nadavo11 (nadavoteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import lpips, imageio, numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from pathlib import Path, PurePath
import torch, time, csv
import csv, json, torch, lpips, imageio, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import csv, torch, lpips, imageio, wandb, numpy as np
from pathlib import Path
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

project="gs-red-truck"
run = wandb.init(project=project, resume="allow")

def eval_model_wandb(model, val_cams, iter_num, run = run,
                     project="gs-red-truck", device="cuda"):

    run = wandb.init(project=project, resume="allow")

    lpips_fn = lpips.LPIPS(net="vgg").to(device).eval()
    model = model.to(device).eval()

    rows, psnr_all, ssim_all, lp_all = [], [], [], []

    with torch.no_grad():
        for cam in val_cams:
            cam = cam.to(device)
            pred = model.forward(cam, active_sh_degree=10).clamp(0, 1)
            gt   = cam.image.float() / 255.0

            psnr = peak_signal_noise_ratio(gt.cpu().numpy().transpose(1,2,0),
                                           pred.cpu().numpy().transpose(1,2,0),
                                           data_range=1.0)
            ssim = structural_similarity(gt.cpu().numpy().transpose(1,2,0),
                                         pred.cpu().numpy().transpose(1,2,0),
                                         channel_axis=2, data_range=1.0)
            lp   = lpips_fn(pred.unsqueeze(0), gt.unsqueeze(0)).item()

            rows.append([cam.image_path, psnr, ssim, lp])
            psnr_all.append(psnr); ssim_all.append(ssim); lp_all.append(lp)

    # ---- Log aggregate numbers for this iteration -------------
    run.log({
        "iter":  iter_num,
        "val/psnr":  np.mean(psnr_all),
        "val/ssim":  np.mean(ssim_all),
        "val/lpips": np.mean(lp_all),
    }, step=iter_num)

    # ---- Upload preview (first val view) ----------------------
    first_cam = val_cams[0].to(device)
    pred = model.forward(first_cam, active_sh_degree=10).clamp(0, 1).cpu().detach().numpy().transpose(1,2,0)
    gt   = (first_cam.image.cpu().numpy().transpose(1,2,0) )
    err  = np.abs(pred - gt) / np.maximum(np.abs(pred - gt).max(), 1e-8)

    preview = np.concatenate([gt, pred, err], axis=1)
    wandb_preview = wandb.Image((preview * 255).astype(np.uint8),
                                caption=f"iter {iter_num}")
    run.log({"preview": wandb_preview}, step=iter_num)

    # ---- Save model → artifact (versioned in W&B) -------------
    ply_path = Path(f"model_iter-{iter_num:04d}.ply")
    model.save_ply("./"+str(ply_path))
    art = wandb.Artifact("gaussian_cloud", type="model",
                         description="Ply dump each val interval")
    art.add_file(str(ply_path))
    run.log_artifact(art)

    # ---- Optional: frame-level table (one row per view) -------
    tbl = wandb.Table(columns=["frame", "psnr", "ssim", "lpips"], data=rows)
    run.log({f"frame_metrics/iter_{iter_num}": tbl}, step=iter_num)

    print(f"✅ logged iter {iter_num} to W&B")
    run.finish()
eval_model_wandb(model = model, val_cams = val_cams, iter_num = 42,project="gs-red-truck"  )


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth
✅ logged iter 42 to W&B


iter,▁
val/lpips,▁
val/psnr,▁
val/ssim,▁
iter,42
val/lpips,0.53319
val/psnr,6.05453
val/ssim,0.01401


In [ ]:
TORCH_USE_CUDA_DSA = True

In [ ]:
# import torch, random
# from gs.core.GaussianModel import GaussianModel

# root = "red_truck_GS"
# cams_dict, _ = load(root)
# cam = next(iter(cams_dict)).to("cuda")

# model = GaussianModel(sh_degree=0).to("cuda")
# # 1 tiny Gaussian so memory use is negligible
# model.positions.data   = torch.zeros(1,3, device="cuda")
# model.scales.data      = torch.ones (1,3, device="cuda")*0.01
# model.opacities.data   = torch.ones (1,1, device="cuda")*0.1
# model.sh_coefficients_0.data[:] = 0.5

torch.cuda.empty_cache()
torch.cuda.synchronize()

try:
    with torch.no_grad():
        frame = model.forward(cam, active_sh_degree=0)
    print("✅ forward ok, shape:", frame.shape)
except RuntimeError as e:
    print("❌ still crashes, so the bug is kernel/driver")
    raise
